In [4]:
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import keras
import resnet
import keras.backend as K
import tensorflow as tf

from skimage import feature as ft
from skimage import transform
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
import FinalNetwork 
import MyNetWork 


Using TensorFlow backend.


In [2]:
def featureselected(img):
    hog_features = ft.hog(img,  # input image
                  orientations=8,  # number of bins
                  pixels_per_cell=(20,20), # pixel per cell
                  cells_per_block=(2,2), # cells per blcok
                  block_norm = 'L2-Hys', #  block norm : str {‘L1’, ‘L1-sqrt’, ‘L2’, ‘L2-Hys’}
                  transform_sqrt = True, # power law compression (also known as gamma correction)
                  feature_vector=True, # flatten the final vectors
                  visualize=False) # return HOG map
    
    temp = img.reshape(128,128)
    
    lbp_features = ft.local_binary_pattern(temp,  # input image
                                   P=8,  # Number of circularly symmetric neighbour set points
                                   R=1.0, # Radius of circle
                                   method='default') # {'default', 'ror', 'uniform', 'var'}
    
    haar_features = ft.haar_like_feature(temp,  # input image
                                0,  # Row-coordinate of top left corner of the detection window.
                                0,  # Column-coordinate of top left corner of the detection window.
                                5,  # Width of the detection window.
                                5,  # Height of the detection window.
                                feature_type=None # The type of feature to consider:
                                )
    
    _lbp = lbp_features.flatten()
    
    train = np.concatenate((hog_features, _lbp, haar_features),axis=0)
    return train

In [3]:
def trainGenerator(x_train_arr,f_train_arr,y_train_arr):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''

    
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images
    
    
    genX1 = datagen.flow(x_train_arr,y_train_arr, batch_size=batch_size)
#    genX2 = datagen.flow(f_train_arr, batch_size=batch_size)
#     while True:
#         X1i = genX1.next()
#         X2i = genX2 .next()
#         print (X1i[0].shape)
#         print (X2i.shape)
#         print (X1i[1].shape)
#         yield ([X1i[0], X2i ], X1i[1])
    while True:
        X1i = genX1.next()
        #X2i = genX2 .next()
        features = []
        for i in range(len(X1i[0])):
            features.append(featureselected(X1i[0][i]).reshape(1, 17472, 1))
        features = np.array(features)

        yield ([X1i[0], features ], X1i[1])   

    

        
def testGenerator(x_test_arr,f_test_arr,y_test_arr):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    datagen = ImageDataGenerator(rotation_range=90, 
                                   width_shift_range=0.05, 
                                   height_shift_range=0.05,
                                   zoom_range=0.1)




    genX1 = datagen.flow(x_test_arr,y_test_arr, seed=7, batch_size=batch_size)
    genX2 = datagen.flow(f_test_arr, seed=7, batch_size=batch_size)
    while True:
        X1i = genX1.next()
        X2i = genX2.next()

        yield ([X1i[0], X2i ], X1i[1])

In [4]:
def incorrect_index(predicts, real_labels):
    results = np.empty(0)
    for i in range(predicts.shape[0]):
        if real_labels[i][0] == 1:
            if predicts[i][0] < 0.3:
                results = np.append(results, i)
        else:
            if predicts[i][1] < 0.3:
                results = np.append(results, i)
    
    return results

def delete_index(incorret_lists):
    counts = np.zeros(206, dtype=int)
    for invalid in incorrect_lists:
        for i in invalid:
            counts[int(i)] += 1
    
    results = np.empty(0)
    for i in range(206):
        if counts[i] >= 5:
            results = np.append(results, i)
    return results

In [5]:
num_classes = 2

# input image dimensions
img_rows, img_cols = 128, 128
input_shape = (img_rows, img_cols, 1)

Input = np.load('Data/final_data.npz')

data = Input["data"]
label = Input["label"]

feature = []
for i in range(data.shape[0]):
    feature.append(featureselected(data[i]))
feature = np.array(feature)
print (feature.shape)
feature.resize(feature.shape[0],1, feature.shape[1],1)

print (data.shape)
print (label.shape)
print (feature.shape)

(1022, 17472)
(1022, 128, 128, 1)
(1022, 2)
(1022, 1, 17472, 1)


In [6]:
def unison_shuffled_copies(a, b, c, rl):
    assert len(a) == len(b)
    assert len(a) == len(c)
    r=list(range(len(a)))
#     random.shuffle(r,lambda:rl)
    random.shuffle(r,lambda:rl)
    p=np.array(r)
    #p = np.random.permutation(len(a))
    return a[p], b[p], c[p]

data,label,feature=unison_shuffled_copies(data,label,feature,0.5)

print(feature.shape)

(1022, 1, 17472, 1)


In [7]:
for i in range(5):
    
    print('-------------------------------------------'+str(i)+'--------------------------------------------------')
    splitpoint1 = int(i * 204)
    splitpoint2 = int((i + 1) * 204)
    if(i==4):
        splitpoint2 = data.shape[0]
    train_x = np.vstack((data[:splitpoint1], data[splitpoint2:]))
    train_f = np.vstack((feature[:splitpoint1], feature[splitpoint2:]))
    train_y = np.vstack((label[:splitpoint1], label[splitpoint2:]))
    val_x = data[splitpoint1:splitpoint2]
    val_f = feature[splitpoint1:splitpoint2]
    val_y = label[splitpoint1:splitpoint2]
    incorrect_list=[]
    print(val_x.shape)

-------------------------------------------0--------------------------------------------------
(204, 128, 128, 1)
-------------------------------------------1--------------------------------------------------
(204, 128, 128, 1)
-------------------------------------------2--------------------------------------------------
(204, 128, 128, 1)
-------------------------------------------3--------------------------------------------------
(204, 128, 128, 1)
-------------------------------------------4--------------------------------------------------
(206, 128, 128, 1)


In [8]:
batch_size = 8
nb_classes = 2
nb_epoch = 150
data_augmentation = True
deleteIndexes = np.zeros(0)

for i in range(5):
    
    print('-------------------------------------------'+str(i)+'--------------------------------------------------')
    splitpoint1 = int(i * 204)
    splitpoint2 = int((i + 1) * 204)
    if(i==4):
        splitpoint2 = data.shape[0]
    train_x = np.vstack((data[:splitpoint1], data[splitpoint2:]))
    train_f = np.vstack((feature[:splitpoint1], feature[splitpoint2:]))
    train_y = np.vstack((label[:splitpoint1], label[splitpoint2:]))
    val_x = data[splitpoint1:splitpoint2]
    val_f = feature[splitpoint1:splitpoint2]
    val_y = label[splitpoint1:splitpoint2]
    incorrect_lists=[]
    print(val_x.shape)
    
    for j in range(5):
        K.clear_session()
        tf.reset_default_graph()
    
        lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
        #csv_logger = CSVLogger('RoomsLogger/Room' + str(i)+'_'+str(j) + '.csv')
        early_stopper = EarlyStopping(min_delta=0.001, patience=10)
        #checkpoint = ModelCheckpoint('RoomModel/Model_'+str(i)+'_'+str(j)+'.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    
        model =  FinalNetwork.NetBuilder.finalNet(input_shape = (128, 128, 1), feature_shape = (1,17472, 1), num_outputs = nb_classes, sampling_size = (128, 128))
        model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
        
        
        trainGene = trainGenerator(train_x,train_f,train_y)

        if not data_augmentation:
            print('Not using data augmentation.')
        
        else:
            print('Using real-time data augmentation.')
            model.fit_generator(trainGene,
                                steps_per_epoch=train_x.shape[0] // batch_size,
                                validation_data=([val_x, val_f], val_y),validation_steps = val_x.shape[0]//batch_size,
                                epochs=nb_epoch, verbose=0, max_q_size=100,
                                callbacks=[lr_reducer, early_stopper])
        
        score = model.evaluate([val_x, val_f], val_y, verbose=0)
        print('Test score:', score[0])
        print('Test accuracy:', score[1])
    
        predictions = model.predict([val_x, val_f], verbose=0)
    
        incrt = incorrect_index(predictions, val_y)
        incorrect_lists.append(incrt)
        
    useless_index = delete_index(incorrect_lists)
    print("delete count:", useless_index.shape[0])
    deleteIndexes = np.append(deleteIndexes, 204*i+useless_index)

-------------------------------------------0--------------------------------------------------
(204, 128, 128, 1)
Using real-time data augmentation.


/home/ilab/.conda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:49: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., verbose=0, validation_data=([array([[..., steps_per_epoch=102, epochs=150, callbacks=[<keras.ca..., max_queue_size=100, validation_steps=25)`


('Test score:', 7.861898702733657)
('Test accuracy:', 0.5147058826451208)
Using real-time data augmentation.
('Test score:', 0.6230512156206018)
('Test accuracy:', 0.7156862733410854)
Using real-time data augmentation.
('Test score:', 0.588016459754869)
('Test accuracy:', 0.7254901949097129)
Using real-time data augmentation.
('Test score:', 0.6702137250526279)
('Test accuracy:', 0.6225490184391246)
Using real-time data augmentation.
('Test score:', 0.6909128696310753)
('Test accuracy:', 0.6225490196078431)
('delete count:', 5)
-------------------------------------------1--------------------------------------------------
(204, 128, 128, 1)
Using real-time data augmentation.
('Test score:', 0.6913022784625783)
('Test accuracy:', 0.651960785482444)
Using real-time data augmentation.
('Test score:', 0.6751316587130228)
('Test accuracy:', 0.7009803921568627)
Using real-time data augmentation.
('Test score:', 0.6180680475982965)
('Test accuracy:', 0.7254901960784313)
Using real-time data au

In [9]:
print(deleteIndexes)

[  46.   50.   59.   72.  193.  258.  275.  361.  416.  439.  468.  504.
  519.  561.  572.  575.  610.  676.  689.  694.  767.  848.  872.  967.
 1003.]


In [10]:
selected_data = []
selected_label = []

for i in range(data.shape[0]):
    if np.isin(i, deleteIndexes, invert=True):
        selected_data.append(data[i])
        selected_label.append(label[i])
        
selected_data = np.array(selected_data)
selected_label = np.array(selected_label)
        
print(selected_data.shape)
print(selected_label.shape)

(997, 128, 128, 1)
(997, 2)


In [12]:
np.savez('Data/experimentData.npz', data=np.array(selected_data), label=np.array(selected_label))

In [7]:


Input = np.load('Data/experimentData.npz')

data = Input["data"]
label = Input["label"]

num_0 = 0
for i in label:
    if i[0]==1:
        num_0+=1
        
print(num_0)

503
